<a href="https://colab.research.google.com/github/nandanjunior/My-Marine-Route-AI/blob/main/Fine_Tuning_(Test_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.2 MB/

In [11]:
!pip install triton==2.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.3.0
    Uninstalling triton-3.3.0:
      Successfully uninstalled triton-3.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.7.0 requires triton==3.3.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 2.3.1 which is incompatible.
unsloth 2025.6.12 requires triton>=3.0.0; platform_system == "Linux", but you have triton 2.3.1 which is incompatible.
unsloth-zoo 2025.6.8 requires triton>=3.0.0; platform_system == "Linux", but you have triton 2.3.1 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.0 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.


In [1]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [2]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
import torch
import json
import pandas as pd
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model_name = "unsloth/llama-3-8b-instruct"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 2.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
tokenizer.pad_token = tokenizer.unk_token

In [5]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.6.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from google.colab import files
import json

# Upload .robot files
uploaded = files.upload()

# --- Helpers ---
def extract_test_case_section(content):
    lines = content.splitlines()
    test_case_lines = []
    inside_test_section = False
    for line in lines:
        if line.strip().startswith("*** Test Cases ***"):
            inside_test_section = True
            continue
        if inside_test_section:
            if line.strip().startswith("***"):
                break
            test_case_lines.append(line.rstrip())
    return test_case_lines

def extract_structured_test_cases(lines):
    test_cases = []
    current_case = None
    for line in lines:
        if not line.strip():
            continue
        if not line.startswith(" ") and not line.startswith("#"):
            if current_case:
                test_cases.append(current_case)
            current_case = {"name": line.strip(), "body": [], "doc": ""}
        elif current_case:
            current_case["body"].append(line.rstrip())
            if "[Documentation]" in line:
                doc_text = line.split("[Documentation]")[-1].strip()
                if doc_text.startswith("..."):
                    doc_text = doc_text[3:].strip()
                current_case["doc"] = doc_text
    if current_case:
        test_cases.append(current_case)
    return test_cases

def to_finetune_format(test_cases):
    dataset = []
    for case in test_cases:
        # Build instruction more intelligently
        instruction = case["doc"].strip() if case["doc"] else f"Create a Robot Framework test case for: {case['name']}"

        # Build output: full test case block
        output_lines = ["*** Test Cases ***", case["name"]] + case["body"]
        output = "\n".join(output_lines)

        dataset.append({
            "text": f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{instruction}\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n{output}\n<|eot_id|>"
        })
    return dataset

# --- Main Processing ---
all_test_cases = []

for filename in uploaded:
    content = uploaded[filename].decode("utf-8")
    test_case_lines = extract_test_case_section(content)
    structured = extract_structured_test_cases(test_case_lines)
    all_test_cases.extend(structured)

# Convert to Unsloth-compatible format
finetune_data = to_finetune_format(all_test_cases)

# Save as JSONL for training
output_file = "robot_framework_finetune_dataset.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for item in finetune_data:
        f.write(json.dumps(item) + "\n")

# Download
files.download(output_file)

Saving Full_Test.robot to Full_Test (1).robot
Saving safety_Full_Test.robot to safety_Full_Test (1).robot


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
from datasets import load_dataset

# Load dataset from the JSONL file
dataset = load_dataset("json", data_files="robot_framework_finetune_dataset.jsonl", split="train")

# Optional: show a few examples
dataset[0]


Generating train split: 0 examples [00:00, ? examples/s]

{'text': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nCheck on general voltage reading\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n*** Test Cases ***\nEnvD_SysQual_PowerManagementSubsystem_General_VoltageMonitoringWithInterbladeSignal\n    [Documentation]    Check on general voltage reading\n    [Tags]    POWER    FID-1275341,2655699    TCID-195840    FULLTEST\n    Log To Console    Hi\n    prj.sut.disconnect.dlt\n    bits_platform.Output Set Analog Signal Value    Battery    0\n    prj.sut.connect.dlt\n    prj.sut.can.initandloadconfiguration\n    prj.sut.can.simulation.Start\n    prj.sut.can.simulation.PowerBladeON\n    prj.sut.power.add.voltage.to.dictionary.check    16.4\n<|eot_id|>'}

In [8]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,  # Enable this to speed up training unless you're using long sequences
    args=SFTConfig(
        per_device_train_batch_size=1,               # Lower batch size due to 8B model on T4
        gradient_accumulation_steps=16,              # Simulates effective batch size of 16
        warmup_ratio=0.1,                            # 10% warmup for stable loss
        max_steps=100,                               # Adjust as needed (or use num_train_epochs)
        logging_steps=1,                             # Log every step
        learning_rate=5e-5,                          # Lower LR improves stability for instruction-tuned LLMs
        weight_decay=0.01,
        lr_scheduler_type="cosine",                  # Cosine often better than linear
        optim="adamw_torch_fused",                   # Recommended if supported by GPU
        seed=3407,
        output_dir="outputs",
        report_to="none",
        save_strategy="steps",
        save_steps=50,                               # Save mid-training checkpoint
        save_total_limit=2,                          # Limit checkpoints
        bf16=False,                                  # T4 doesn't support bfloat16
        fp16=True,                                   # Use fp16 with bitsandbytes 4-bit model
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/688 [00:00<?, ? examples/s]

In [9]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 688 | Num Epochs = 3 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 167,772,160 of 8,000,000,000 (2.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.485100
2,3.282300
3,3.695800
4,3.071100
5,2.422600
6,2.585500
7,2.505700
8,2.722600
9,2.399400
10,1.816200


Step,Training Loss
1,2.485100
2,3.282300
3,3.695800
4,3.071100
5,2.422600
6,2.585500
7,2.505700
8,2.722600
9,2.399400
10,1.816200


In [15]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Check on general voltage reading"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Check on general voltage reading<|eot_id|><|start_header_id|>assistant<|end_header_id|>

*** Test Cases ***
EnvD_SysQual_Diagnostics_VoltageMonitoring_VoltageMonitoring(GeneralVoltage)
    [Documentation]    Check on general voltage reading
    [Tags]    CustomerDiagnostic    TCID-272574    FULLTEST	FID-1151029,1151030,1151031,1151032,1151033
    prj.sut.can.diag.send    ${sut.diag.Extended_Session_Req}
    prj.sut.diag.compare    ${sut.diag.Extended_Session_Resp}
    prj.sut.can.diag.send    ${sut.diag.GeneralVoltageMonitoring_Req}
    prj.sut.diag.compare    ${sut.diag.GeneralVoltageMonitoring_Resp}
<|eot_id|>
